In [ ]:
from atproto import Client 
from dotenv import dotenv_values 
import asyncio 
import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt 
import json
import atproto_client.models as models

dotenv = dotenv_values('.env')

client = Client() 
client.login(dotenv['USERNAME'], dotenv['PASSWORD']);

In [ ]:
response = client.app.bsky.feed.search_posts(
    params=models.AppBskyFeedSearchPosts.Params(
        q='#example',  
    )
)

for post in response.posts:
    print(post.author.handle, post.record.text)